Пример программы **Python**, которая выполняет семантическую сегментацию видео с использованием предварительно обученной модели и предоставляет текстовое описание каждого сегментированного объекта. В этом примере используется архитектура **DeepLabv3+** для семантической сегментации и загружается предварительно обученная модель из зоопарка моделей **TensorFlow**.

Для начала установим необходимые библиотеки:

In [ ]:
!pip install tensorflow==2.8.0

In [ ]:
!pip install opencv-python-headless==4.1.2.30

In [26]:
import cv2
import numpy as np
import urllib.request
import tensorflow as tf

# Загрузите предварительно обученную модель DeepLabv3+ из зоопарка моделей TensorFlow
model_url = 'http://download.tensorflow.org/models/deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz'
model_path = '/content/deeplabv3_model.tar.gz'
urllib.request.urlretrieve(model_url, model_path)

# Извлеките файлы модели
import tarfile
with tarfile.open(model_path, 'r:gz') as tar:
    tar.extractall()
model_dir = './deeplabv3_mnv2_pascal_trainval/'

# Загрузите предварительно обученную модель семантической сегментации
graph = tf.Graph()
with tf.compat.v1.gfile.FastGFile(model_dir + 'frozen_inference_graph.pb', 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def, name='')

# Определите метки классов для сегментации
class_labels = ['фон', 'самолет', 'велосипед', 'птица', 'лодка', ...]  # Добавьте больше этикеток в соответствии с вашей моделью

# Загрузите видео
video_path = '/content/path_to_video.mp4'
cap = cv2.VideoCapture(video_path)

# Получить свойства видео
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Создайте объект VideoWriter для сохранения обработанных кадров
output_path = '/content/path_to_video1.mp4'
fourcc = cv2.VideoWriter_fourcc(*'MP4V')  # Используйте кодек MP4V, XVID, MJPG, H264. По вашу усмотрению
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Создайте сеанс TensorFlow
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
with tf.compat.v1.Session(graph=graph, config=config) as sess:
    # Получить входные и выходные тензоры
    input_tensor = sess.graph.get_tensor_by_name('ImageTensor:0')
    output_tensor = sess.graph.get_tensor_by_name('SemanticPredictions:0')

    # Прокрутите каждый кадр в видео
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Предварительно обработайте кадр
        frame = cv2.resize(frame, (513, 513))
        frame = frame / 255.0
        frame = np.expand_dims(frame, axis=0)

        # Выполнение семантической сегментации
        predicted_mask = sess.run(output_tensor, feed_dict={input_tensor: frame})
        predicted_mask = np.argmax(predicted_mask, axis=-1)
        predicted_mask = predicted_mask[0]

        # Создание текстовых описаний для каждого сегментированного объекта
        for label in np.unique(predicted_mask):
            if label == 0:
                continue  # Пропустить фоновую метку

            mask = np.uint8(predicted_mask == label)
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            object_area = cv2.contourArea(contours[0])

            # Отфильтруйте мелкие объекты
            if object_area < 500:
                continue

            # Сгенерировать описание объекта
            object_label = class_labels[label]
            object_description = f"В {object_label} присутствует в кадре."

            # Отображение описания объекта на рамке
            cv2.putText(frame, object_description, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Сохраните обработанный кадр в новый видеофайл
        out.write(np.uint8(frame))

    # Отпустите захват видео и закройте окна
    cap.release()
    out.release()
    cv2.destroyAllWindows()


Вот пошаговое описание алгоритма на основе предоставленного кода:

1. Установите пакеты **TensorFlow** и **OpenCV**: этот блок кода устанавливает необходимые пакеты с помощью **pip**.

2. Загрузите предварительно обученную модель **DeepLabv3+**: код загружает модель **DeepLabv3+** из зоопарка моделей **TensorFlow**, используя указанный URL-адрес, и сохраняет ее по указанному пути к модели.

3. Извлеките файлы модели: код извлекает загруженные файлы модели из архива **tar.gz** с помощью модуля **tarfile** и сохраняет их в каталоге **model_dir**.

4. Загрузите предварительно обученную модель семантической сегментации: код загружает определение графа предварительно обученной модели из файла **Frozen_inference_graph.pb** и устанавливает его в качестве графа по умолчанию в **TensorFlow**.

5. Определите метки классов для сегментации: код определяет список меток классов, используемых для семантической сегментации. Вы можете добавить в список дополнительные метки в зависимости от конкретной модели.

6. Загрузите видео: код загружает входное видео из указанного пути к видео с помощью класса **OpenCV VideoCapture**.

7. Получить свойства видео: код извлекает ширину, высоту и количество кадров в секунду **(fps)** видео, используя свойства захвата видео **OpenCV**.

8. Создать объект **VideoWriter**: код создает объект **VideoWriter** для сохранения обработанных кадров в новый видеофайл. Он указывает выходной путь, кодек **MP4V**, частоту кадров и размер кадра.

9. Создайте сеанс **TensorFlow**: код создает сеанс **TensorFlow** с загруженным графом и конфигурацией графического процессора. Это позволяет динамически распределять память графического процессора с помощью **allow_growth**.

10. Выполнение семантической сегментации каждого кадра: код считывает каждый кадр из видео с помощью метода **cap.read()** внутри цикла **while**. Если кадр успешно прочитан, он переходит к его обработке.

11. Предварительно обработайте кадр: код изменяет размер кадра до требуемого входного размера модели **(513x513)**, нормализует значения пикселей от **0** до **1** и расширяет размеры, чтобы они соответствовали входной форме модели.

12. Выполнение семантической сегментации: код пропускает предварительно обработанный кадр через загруженную модель, чтобы получить предсказанную маску сегментации. Он выбирает наиболее вероятную метку для каждого пикселя, находя индекс максимального значения по последней оси.

13. Создание текстовых описаний для сегментированных объектов: код перебирает уникальные метки в предсказанной маске и отфильтровывает фоновую метку **(0)**. Для каждой оставшейся метки он создает бинарную маску, находит контуры и вычисляет площадь объекта. Небольшие объекты площадью менее **500** пикселей пропускаются. Он генерирует описание для каждого объекта на основе метки и сохраняет его в **object_description**.

14. Отображение описаний объектов на фрейме: код накладывает созданные описания объектов на фрейм с помощью функции **OpenCV putText**, указывая положение, шрифт, цвет и толщину.

15. Сохраните обработанный кадр в новый видеофайл: код записывает обработанный кадр в выходной видеофайл с помощью метода **out.write()**.

16. Освобождение ресурсов и закрытие окон. После обработки всех кадров код освобождает объекты захвата и записи видео и закрывает все открытые окна.

Этот алгоритм выполняет семантическую сегментацию каждого кадра входного видео, генерирует описания объектов и сохраняет обработанные кадры с описаниями в новый видеофайл.